In [62]:
from __future__ import print_function
import tarfile

from os import listdir,remove
from os.path import exists, join, basename

from six.moves import urllib
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torch.utils.data as data

from math import log10
import sys
import time
import numpy as np
#import argparse
import easydict      #rd
import cv2

#  Prepare Data                                 <br>
Download data if can't find it in local directory           <br>
Get training set from  './dataset/HR_img_train', get testset from  './dataset/HR_img_test'     <br>


In [76]:
def download_bsd300(dest=".//dataset"):
    output_image_dir = join(dest, "BSDS300/images")

    if not exists(output_image_dir):
        url = "http://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/BSDS300-images.tgz"
        print("downloading url ", url)

        data = urllib.request.urlopen(url)

        file_path = join(dest, basename(url))
        with open(file_path, 'wb') as file:
            print("Extracting data")
        with tarfile.open(file_path) as tar:
            for item in tar:
                tar.extract(item, dest)

        remove(file_path)

    return output_image_dir    #rd


def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)    #rd    what does this mean?

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])  #rd


def load_img(filepath):
    img = Image.open(filepath).convert('YCbCr')
    y, _, _ = img.split()
    return y


class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, input_transform=None, target_transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]

        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        input_image = load_img(self.image_filenames[index])
        target = input_image.copy()
        # print(target.size)
        if self.input_transform:
            input_image = self.input_transform(input_image)
        else:
            transi = Compose([
                            Resize((input_image.size[1] // 4, input_image.size[0] // 4), Image.BICUBIC),
                            ToTensor()
                            ])
            input_image = transi(input_image)
        if self.target_transform:
            target = self.target_transform(target)
        else:
            transt = Compose([
                ToTensor()
            ])
            target = transt(target)

        # print(input_image.size(), target.size())
        return input_image, target

    def __len__(self):
        return len(self.image_filenames)

def input_transform(crop_size, upscale_factor):   #get the low resolution input by original image.  This is a function, return can be viewed
                                                  # as a set of parameters import into DatasetFromFolder.input_trainsform and target_transform
    return Compose([
        CenterCrop(crop_size),                        #get a small size of data, Corp with less pieces and resize corp? what does corp mean 
                                                      # in this project? why not whole img
        Resize(crop_size // upscale_factor),           # Resize is what? any help? a way to down size? is it reliable?
        ToTensor(),
    ])


def target_transform(crop_size):
    return Compose([
        CenterCrop(crop_size),
        ToTensor(),
    ])


def get_training_set(upscale_factor):
    # root_dir = download_bsd300()
    root_dir = './/dataset//'
    train_dir = join(root_dir, "HR_img_train//")
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(train_dir,
                             input_transform=None,
                             target_transform=None
                             # input_transform=input_transform(crop_size, upscale_factor),
                             # target_transform=target_transform(crop_size))
                             )


def get_test_set(upscale_factor):
    # root_dir = download_bsd300()
    root_dir = './/dataset//'
    test_dir = join(root_dir, "HR_img_test//")
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(test_dir,
                             input_transform=None,
                             target_transform=None
                             # input_transform=input_transform(crop_size, upscale_factor),
                             # target_transform=target_transform(crop_size))
                             )

# Visualize Training precedure

In [64]:
TOTAL_BAR_LENGTH = 80
LAST_T = time.time()
BEGIN_T = LAST_T


def progress_bar(current, total, msg=None):
    global LAST_T, BEGIN_T
    if current == 0:
        BEGIN_T = time.time()  # Reset for new bar.

    current_len = int(TOTAL_BAR_LENGTH * (current + 1) / total)
    rest_len = int(TOTAL_BAR_LENGTH - current_len) - 1

    sys.stdout.write(' %d/%d' % (current + 1, total))
    sys.stdout.write(' [')
    for i in range(current_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    current_time = time.time()
    step_time = current_time - LAST_T
    LAST_T = current_time
    total_time = current_time - BEGIN_T

    time_used = '  Step: %s' % format_time(step_time)
    time_used += ' | Tot: %s' % format_time(total_time)
    if msg:
        time_used += ' | ' + msg

    msg = time_used
    sys.stdout.write(msg)

    if current < total - 1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()


# return the formatted time
def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    seconds_final = int(seconds)
    seconds = seconds - seconds_final
    millis = int(seconds*1000)

    output = ''
    time_index = 1
    if days > 0:
        output += str(days) + 'D'
        time_index += 1
    if hours > 0 and time_index <= 2:
        output += str(hours) + 'h'
        time_index += 1
    if minutes > 0 and time_index <= 2:
        output += str(minutes) + 'm'
        time_index += 1
    if seconds_final > 0 and time_index <= 2:
        output += str(seconds_final) + 's'
        time_index += 1
    if millis > 0 and time_index <= 2:
        output += str(millis) + 'ms'
        time_index += 1
    if output == '':
        output = '0ms'
    return output                      #rd

#  Built CNN Model 

In [84]:
class Net(torch.nn.Module):
    def __init__(self, num_channels, upscale_factor, d=64, s=12, m=4):
        super(Net, self).__init__()

        self.first_part = nn.Sequential(nn.Conv2d(in_channels=num_channels, out_channels=d, kernel_size=5, stride=1, padding=2),
                                        nn.PReLU())

        self.layers = []
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=d, out_channels=s, kernel_size=1, stride=1, padding=0), nn.PReLU()))
        for _ in range(m):

            # self.layers.append(nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, stride=1, padding=1))      # Normal conv

            self.layers.append(nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, stride=1, padding=1, groups=s))      # MobileNet conv
            self.layers.append(nn.Conv2d(in_channels=s, out_channels=s, kernel_size=1, stride=1, padding=0))

        self.layers.append(nn.PReLU())
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=s, out_channels=d, kernel_size=1, stride=1, padding=0), nn.PReLU()))

        self.mid_part = torch.nn.Sequential(*self.layers)

        self.last_part = nn.ConvTranspose2d(in_channels=d, out_channels=num_channels, kernel_size=9, stride=upscale_factor, padding=3, output_padding=1)

    def forward(self, x):
        out = self.first_part(x)
        out = self.mid_part(out)
        out = self.last_part(out)
        return out

    def weight_init(self, mean=0.0, std=0.02):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(mean, std)
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                m.weight.data.normal_(0.0, 0.0001)
                if m.bias is not None:
                    m.bias.data.zero_()

class FSRCNNTrainer(object):
    def __init__(self, config, training_loader, testing_loader):
        super(FSRCNNTrainer, self).__init__()
        self.CUDA = torch.cuda.is_available()
        self.device = torch.device('cuda' if self.CUDA else 'cpu')
        self.model = None
        self.lr = config.lr
        self.nEpochs = config.nEpochs
        self.criterion = None
        self.optimizer = None
        self.scheduler = None
        self.seed = config.seed
        self.upscale_factor = config.upscale_factor
        self.training_loader = training_loader
        self.testing_loader = testing_loader
        self.testpsnr = 0

    def build_model(self):
        self.model = Net(num_channels=1, upscale_factor=self.upscale_factor).to(self.device)
        self.model.weight_init(mean=0.0, std=0.2)
        self.criterion = torch.nn.MSELoss()
        torch.manual_seed(self.seed)

        if self.CUDA:
            torch.cuda.manual_seed(self.seed)
            cudnn.benchmark = True
            self.criterion.cuda()

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[50, 75, 100], gamma=0.5)  # lr decay

    def save_model(self):
        model_out_path = "model_path.pth"
        torch.save(self.model, model_out_path)
        print("Checkpoint saved to {}".format(model_out_path))

    def train(self):
        self.model.train()
        train_loss = 0
        for batch_num, (data, target) in enumerate(self.training_loader):
            data, target = data.to(self.device), target.to(self.device)
            self.optimizer.zero_grad()
            loss = self.criterion(self.model(data), target)
            train_loss += loss.item()
            # print(train_loss / (batch_num + 1))
            loss.backward()
            self.optimizer.step()
            #print('batch_num:', batch_num, '/', len(self.training_loader), 'Loss:', train_loss / (batch_num + 1))
            progress_bar(batch_num, len(self.training_loader), 'Loss: %.4f' % (train_loss / (batch_num + 1)))

        print("    Average Loss: {:.4f}".format(train_loss / len(self.training_loader)))

    def test(self):
        self.model.eval()
        avg_psnr = 0

        with torch.no_grad():
            for batch_num, (data, target) in enumerate(self.testing_loader):
                data, target = data.to(self.device), target.to(self.device)
                prediction = self.model(data)
                mse = self.criterion(prediction, target)
                psnr = 10 * log10(1 / mse.item())
                avg_psnr += psnr
        print('test PSNR:', avg_psnr / len(self.testing_loader))
                # progress_bar(batch_num, len(self.testing_loader), 'PSNR: %.4f' % (avg_psnr / (batch_num + 1)))

        save_path = './saved_models'
        if avg_psnr / len(self.testing_loader) > self.testpsnr:
            self.testpsnr = avg_psnr / len(self.testing_loader)
            folder = os.path.exists(save_path)
            if not folder:
                os.makedirs(save_path)
                print('create folder to save models')
            torch.save(self.model, save_path + '/model_' + str(self.testpsnr))
            print('model saved ......')
        print("    Average PSNR: {:.4f} dB".format(avg_psnr / len(self.testing_loader)))

    def run(self):
        self.build_model()
        print('Total parameters:', sum(p.numel() for p in self.model.parameters()))
        print('Total trainable parameters:', sum(p.numel() for p in self.model.parameters() if p.requires_grad))
        for epoch in range(1, self.nEpochs + 1):
            print("\n===> Epoch {} starts:".format(epoch))
            self.train()
            self.test()
            self.scheduler.step(epoch)
            
 #           check=0
#             if avg_psnr>31:
#                 check=1
#             if avg_psnr<=31& check==1:
#                 self.save_model()
#                 print('model saved with psnr=%f'%avg_psnr)
#                 break
#             if avg_psnr>35:
#                 self.save_model()
#                 print('model saved with psnr=%f'%avg_psnr)
#                 break
#             elif epoch == self.nEpochs:
#                 self.save_model()
#                 print()

# Training model in 'main'

In [184]:
# ===========================================================
# Training settings
# ===========================================================
#parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
# hyper-parameters
#parser.add_argument('--batchSize', type=int, default=4, help='training batch size')
#parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
#parser.add_argument('--nEpochs', type=int, default=20, help='number of epochs to train for')
#parser.add_argument('--lr', type=float, default=0.01, help='Learning Rate. Default=0.01')
#parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')

# model configuration
#parser.add_argument('--upscale_factor', '-uf',  type=int, default=4, help="super resolution upscale factor")

args = easydict.EasyDict({
    'batchSize':1,
    'testBatchSize':1,
   'nEpochs':15,
    'lr':0.001,
    'seed':123,
    'upscale_factor':4,
    'uf':4
    
})


def main():
    # ===========================================================
    # Set train dataset & test dataset
    # ===========================================================
    print('===> Loading datasets')
    train_set = get_training_set(args.upscale_factor)
    test_set = get_test_set(args.upscale_factor)
    training_data_loader = DataLoader(dataset=train_set, batch_size=args.batchSize, shuffle=True)
    testing_data_loader = DataLoader(dataset=test_set, batch_size=args.testBatchSize, shuffle=False)

    model = FSRCNNTrainer(args, training_data_loader, testing_data_loader)

    model.run()


if __name__ == '__main__':
    main()

===> Loading datasets
Total parameters: 9569
Total trainable parameters: 9569

===> Epoch 1 starts:


KeyboardInterrupt: 

# Output high resolution image and compute PSNR


In [57]:
# output high resolution image to output_dir, with a specified model
def output_image(input_dir,input_model,output_dir):
#     test = easydict.EasyDict({
#         'input':'.//test_image//1.jpg',
#         'model':'model_path.pth',              # the loaded model, model should be saved at project folder './mdoel.pth'
#        'output':'.//test_image//1_generate_2.jpg',    
#     })

# ===========================================================
# input image setting
# ===========================================================
    start = time.time()

    GPU_IN_USE = torch.cuda.is_available()
    img = Image.open(input_dir).convert('YCbCr')
    y, cb, cr = img.split()

# ===========================================================
# model import & setting
# ===========================================================
    device = torch.device('cuda' if GPU_IN_USE else 'cpu')
    model = torch.load(input_model, map_location=lambda storage, loc: storage)
    model = model.to(device)
    data = (ToTensor()(y)).view(1, -1, y.size[1], y.size[0])
    data = data.to(device)

    if GPU_IN_USE:
        cudnn.benchmark = True


# ===========================================================
# output and save image
# ===========================================================
    out = model(data)
    out = out.cpu()
    out_img_y = out.data[0].numpy()
    out_img_y *= 255.0
    out_img_y = out_img_y.clip(0, 255)
    out_img_y = Image.fromarray(np.uint8(out_img_y[0]), mode='L')

    out_img_cb = cb.resize(out_img_y.size, Image.BICUBIC)
    out_img_cr = cr.resize(out_img_y.size, Image.BICUBIC)
    out_img = Image.merge('YCbCr', [out_img_y, out_img_cb, out_img_cr]).convert('RGB')

    out_img.save(output_dir)
    
    end = time.time()
    elapsed = end - start
    
    print('output image saved to:', output_dir,'   running time:',elapsed,'s')

# output high resolution image to output_dir, with the current model
def creat_image(input_dir,output_dir):
    start = time.time()
    GPU_IN_USE = torch.cuda.is_available()
    img = Image.open(input_dir).convert('YCbCr')
    y, cb, cr = img.split()
    

 #   model = torch.load(input_model, map_location=lambda storage, loc: storage)
 #   model = model.to(device)
    data = (ToTensor()(y)).view(1, -1, y.size[1], y.size[0])
    data = data.to(device)

    if GPU_IN_USE:
        cudnn.benchmark = True
    out = model(data)
    out = out.cpu()
    out_img_y = out.data[0].numpy()
    out_img_y *= 255.0
    out_img_y = out_img_y.clip(0, 255)
    out_img_y = Image.fromarray(np.uint8(out_img_y[0]), mode='L')

    out_img_cb = cb.resize(out_img_y.size, Image.BICUBIC)
    out_img_cr = cr.resize(out_img_y.size, Image.BICUBIC)
    out_img = Image.merge('YCbCr', [out_img_y, out_img_cb, out_img_cr]).convert('RGB')

    out_img.save(output_dir)
    
    end = time.time()
    elapsed = end - start
    
    print('output image saved to:', output_dir,'   running time:',elapsed,'s')

In [8]:
output_image(input_dir='.//test_image//0.jpg',input_model='model_path2.pth',output_dir='.//test_image//0_generate_3.jpg')

output image saved to: .//test_image//0_generate_3.jpg running time: 2.605011463165283 s


In [22]:
# test of PSNR comparation
def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = max(img1.shape[0],img1.shape[1])
    return 10 * log10(PIXEL_MAX / mse)
#    return 10 * log10(1 / mse)
    
def PSNR(target_dir,original_dir):

    original = cv2.imread(original_dir)
    contrast = cv2.imread(target_dir)
    
    d=psnr(original,contrast)
    print('PSNR=',d)
    return d

In [86]:
image_dir='.//LR_img_0.25'
image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]
original_dir='.//HR_img_4//'

GPU_IN_USE = torch.cuda.is_available()
device = torch.device('cuda' if GPU_IN_USE else 'cpu')
model = torch.load('.//saved_models//model_33.pth', map_location=lambda storage, loc: storage)
model = model.to(device)
   
for i,filename in enumerate (image_filenames):
    img_name=filename.split(os.sep)[-1]
    creat_image(input_dir=filename,output_dir='.//LR_img_output//'+img_name)
  #  print('high resolution by CNN')
   # PSNR('.//HR_img_4//'+filename+'.jpg','.//LR_img_output//'+filename+'.jpg')
   # print('high resolution by BICUIBIC')
   # PSNR('.//HR_img_4//'+filename+'.jpg','.//LR_img_BICUIBIC//'+filename+'.jpg')

output image saved to: .//LR_img_output//new_0.jpg    running time: 1.3434069156646729 s
output image saved to: .//LR_img_output//new_1.jpg    running time: 1.0421850681304932 s
output image saved to: .//LR_img_output//new_10.jpg    running time: 0.8896217346191406 s


In [82]:
original_dir='.//HR_img_4//'
original_filenames = [join(original_dir, x) for x in listdir(original_dir) if is_image_file(x)]
BICUIBIC_dir='.//LR_img_BICUIBIC//'
BICUIBIC_filenames = [join(BICUIBIC_dir, x) for x in listdir(BICUIBIC_dir) if is_image_file(x)]
output_dir='.//LR_img_output//'
output_filenames = [join(output_dir, x) for x in listdir(output_dir) if is_image_file(x)]
face_dir='.//LR_face_img_output//'
face_output_filenames = [join(face_dir, x) for x in listdir(face_dir) if is_image_file(x)]
face_original_dir='.//face_img_4//'
face_filenames = [join(face_original_dir, x) for x in listdir(face_original_dir) if is_image_file(x)]
for i in range (0,50):
    print(output_filenames[i],'high resolution by CNN')
    PSNR(face_filenames[i],face_output_filenames[i])
    #print(BICUIBIC_filenames[i],'high resolution by BICUIBIC')
    #PSNR(original_filenames[i],BICUIBIC_filenames[i])

.//LR_img_output//0.jpg high resolution by CNN
16.27083686424689
.//LR_img_output//1.jpg high resolution by CNN
15.861659877553185
.//LR_img_output//10.jpg high resolution by CNN
18.275048959429615
.//LR_img_output//100.jpg high resolution by CNN
16.040215030275483
.//LR_img_output//101.jpg high resolution by CNN
19.14871264652683
.//LR_img_output//102.jpg high resolution by CNN
17.68598291436126
.//LR_img_output//103.jpg high resolution by CNN
18.355294170829158
.//LR_img_output//104.jpg high resolution by CNN
19.102057471809392
.//LR_img_output//105.jpg high resolution by CNN
11.948134469080374
.//LR_img_output//106.jpg high resolution by CNN
17.453424681585588
.//LR_img_output//107.jpg high resolution by CNN
17.81386131065656
.//LR_img_output//108.jpg high resolution by CNN
18.52581514150305
.//LR_img_output//109.jpg high resolution by CNN
13.165928528115018
.//LR_img_output//11.jpg high resolution by CNN
19.383291008033115
.//LR_img_output//110.jpg high resolution by CNN
19.3563607

In [88]:
PSNR('.\\LR_img_output\\14.jpg','.\\HR_img_4\\14.jpg')

18.404926239417456


18.404926239417456

In [176]:
input_path='.//LR_img_0.25//14.jpg'
output_path='.//test_output//14_1.jpg'
original_path='.\\HR_img_4\\14.jpg'
output_image(input_dir=input_path,input_model='model_path2.pth',output_dir=output_path)
PSNR('.//test_output//14_1.jpg','.\\HR_img_4\\14.jpg')



output image saved to: .//test_output//14_1.jpg    running time: 1.422717571258545 s
18.1453664639979


18.1453664639979

In [177]:
img1=load_img(output_path)

img2=load_img(original_path)


In [181]:
def change_toarray(image):
    row,col =  image.size
    data = np.zeros((row,col),float)
    pixels = image.load()
    for i in range(0,row):
        for j in range(0,col):
            data[i,j] =  pixels[i,j]
    return data
im1=change_toarray(img1)
im2=change_toarray(img2)

In [183]:
a=psnr(im1,im2)
print(a)
PSNR(output_path,original_path)

11.353461568807706
18.1453664639979


18.1453664639979

In [175]:
im1-im2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])